# Análisis de Datos COVID-19 - 2021

## Evaluación del Máster en AI

### 1. Introducción
Este notebook presenta un análisis detallado de los datos COVID-19 del año 2021, utilizando datos almacenados en AWS S3. El objetivo es analizar la evolución de casos confirmados y muertes en diferentes países, con especial énfasis en España y comparativas internacionales.

### 2. Data
Los datos utilizados provienen del dataset COVID del año 2021, accesible a través de AWS Athena. El dataset contiene información diaria sobre:
- Casos confirmados (acumulados)
- Muertes (acumuladas)
- Información por país/región
- Fechas de actualización

### 3. Metodología
El análisis se realizará en tres fases:
1. Consultas mediante AWS Athena para extraer datos relevantes
2. Procesamiento de datos utilizando pandas
3. Visualización mediante matplotlib y seaborn

### 4. Configuración Inicial y Conexión a AWS

Librerías:

In [9]:
# Importar las bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de estilo para las visualizaciones
plt.style.use('default')  # Primero usamos el estilo default
sns.set_theme()  # Aplicamos el tema de seaborn
sns.set_style("whitegrid")  # Configuramos el estilo de la cuadrícula

# Configurar credenciales de AWS (reemplaza con tus credenciales)
import os

#### Conexión AWS

# Consultas SQL realizadas en AWS Athena

## 1. Evolución mensual de casos y muertes en España
Esta consulta nos permite visualizar la progresión temporal de la pandemia en España.
```sql
SELECT year, month,
       MAX(CAST(confirmed as DOUBLE)) as confirmed_cases,
       MAX(CAST(deaths as DOUBLE)) as total_deaths
FROM covid
WHERE country_region = 'Spain'
GROUP BY year, month
ORDER BY year, month;
```

## 2. Top 10 países más afectados
Análisis de los países con mayor número de casos confirmados.
```sql
SELECT country_region,
       MAX(CAST(confirmed as DOUBLE)) as total_confirmed,
       MAX(CAST(deaths as DOUBLE)) as total_deaths
FROM covid
GROUP BY country_region
ORDER BY total_confirmed DESC
LIMIT 10;
```

## 3. Tasa de letalidad por país
Comparativa de la tasa de mortalidad entre países.
```sql
SELECT country_region,
       MAX(CAST(confirmed as DOUBLE)) as total_cases,
       MAX(CAST(deaths as DOUBLE)) as total_deaths,
       (MAX(CAST(deaths as DOUBLE)) / MAX(CAST(confirmed as DOUBLE)) * 100) as mortality_rate
FROM covid
GROUP BY country_region
HAVING MAX(CAST(confirmed as DOUBLE)) > 100000
ORDER BY mortality_rate DESC
LIMIT 15;
```

## 4. Evolución mensual global
Tendencia mundial de casos y muertes por mes.
```sql
SELECT year, month,
       SUM(CAST(confirmed as DOUBLE)) as global_cases,
       SUM(CAST(deaths as DOUBLE)) as global_deaths
FROM covid
GROUP BY year, month
ORDER BY year, month;
```

## 5. Comparativa entre países vecinos
Análisis comparativo entre países de Europa.
```sql
SELECT country_region,
       MAX(CAST(NULLIF(confirmed, '') as DOUBLE)) as total_cases,
       MAX(CAST(NULLIF(deaths, '') as DOUBLE)) as total_deaths,
       MAX(CAST(NULLIF(incident_rate, '') as DOUBLE)) as incident_rate
FROM covid
WHERE country_region IN ('Spain', 'France', 'Portugal', 'Italy', 'Germany')
GROUP BY country_region;
```

## 6. Distribución de casos por continente
Para esto necesitaremos agrupar países por región.
```sql
SELECT
    CASE
        WHEN country_region IN ('China', 'Japan', 'South Korea', 'India') THEN 'Asia'
        WHEN country_region IN ('Spain', 'France', 'Italy', 'Germany', 'UK') THEN 'Europe'
        WHEN country_region IN ('US', 'Canada', 'Mexico') THEN 'North America'
        WHEN country_region IN ('Brazil', 'Argentina', 'Chile') THEN 'South America'
        ELSE 'Other'
    END as continent,
    SUM(CAST(confirmed as DOUBLE)) as total_cases
FROM covid
GROUP BY 1
ORDER BY total_cases DESC;
```

## 7. Países con mayor tasa de incidencia
```sql
SELECT country_region,
       MAX(CAST(NULLIF(incident_rate, '') as DOUBLE)) as max_incident_rate,
       MAX(CAST(NULLIF(confirmed, '') as DOUBLE)) as total_cases
FROM covid
WHERE CAST(NULLIF(incident_rate, '') as DOUBLE) > 0
GROUP BY country_region
ORDER BY max_incident_rate DESC
LIMIT 10;
```

## 8. Análisis de casos activos vs recuperados
```sql
SELECT country_region,
       MAX(CAST(NULLIF(active, '') as DOUBLE)) as active_cases,
       MAX(CAST(NULLIF(recovered, '') as DOUBLE)) as recovered_cases
FROM covid
WHERE CAST(NULLIF(active, '') as DOUBLE) > 0
GROUP BY country_region
ORDER BY active_cases DESC
LIMIT 15;
```

## 9. Evolución semanal en España
```sql
SELECT year, month,
       DATE(SUBSTR(last_update, 1, 10)) as date,
       MAX(CAST(NULLIF(confirmed, '') as DOUBLE)) as confirmed_cases,
       MAX(CAST(NULLIF(deaths, '') as DOUBLE)) as deaths
FROM covid
WHERE country_region = 'Spain'
GROUP BY year, month, DATE(SUBSTR(last_update, 1, 10))
ORDER BY date;
```

## 10. Correlación entre población y casos
```sql
SELECT country_region,
       MAX(CAST(NULLIF(confirmed, '') as DOUBLE)) as total_cases,
       MAX(CAST(NULLIF(incident_rate, '') as DOUBLE)) as incident_rate,
       MAX(CAST(NULLIF(case_fatality_ratio, '') as DOUBLE)) as fatality_ratio
FROM covid
WHERE CAST(NULLIF(incident_rate, '') as DOUBLE) > 0
GROUP BY country_region
ORDER BY total_cases DESC;
```
```

Para cargar imágenes en el markdown de Colab, puedes usar la sintaxis de markdown estándar:
```markdown
![título de la imagen](url_de_la_imagen)
```

